# Creating Crosswalk with DLA Locode, NTD ID and ITP ID

In [1]:
import pandas as pd
from siuba import *
import numpy as np
from calitp import *

In [2]:
from calitp.storage import get_fs
fs = get_fs()

GCS_FILE_PATH = 'gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/'

pd.set_option('display.max_columns', None)

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [3]:
dla = pd.read_excel(f"{GCS_FILE_PATH}locodes_updated7122021.xlsx")

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [4]:
dla = to_snakecase(dla)
dla = dla.rename(columns = {'agency_name':'name', 'agency_locode':'ct_code'})

In [5]:
dla.sample(1)

,ct_code,name,district,county_name,rtpa_name,mpo_name,mpo_locode_fads,active_e76s______7_12_2021_
100,6157,El Dorado County Transportation Commission,3,El Dorado County,El Dorado County Transportation Commission,Sacramento Area Council Of Governments,SACOG,NaN


In [6]:
dla = dla>>select(_.ct_code, _.name, _.district)

### Reading in [Place Names](https://dot.ca.gov/-/media/dot-media/programs/research-innovation-system-information/documents/place-names/2020-place-names-in-california-a11y.pdf)

In [8]:
pl_city = pd.read_excel(f"{GCS_FILE_PATH}2020-place-names-locode.xlsx", sheet_name=0)
pl_county = pd.read_excel(f"{GCS_FILE_PATH}2020-place-names-locode.xlsx", sheet_name=1)

In [9]:
pl_city = to_snakecase(pl_city)
pl_county = to_snakecase(pl_county)

In [10]:
pl_city = pl_city.drop(columns={'unnamed:_1','unnamed:_3','unnamed:_4','unnamed:_6','unnamed:_7', 'date_of_incorporation', 'urban_area_no_'})
pl_county = pl_county.drop(columns={'unnamed:_1','unnamed:_3','unnamed:_4','unnamed:_6','unnamed:_9'})

In [11]:
pl_city = pl_city.rename(columns={'ct_city_code':'ct_code', 'city_name_abbr_':'abbr', 'dist_':'district', 'co_':'county'})
pl_county = pl_county.rename(columns={'ct_co__code':'ct_code', 'county_name':'name', 'co__name_abbr_':'abbr'})

In [12]:
pl_city['organization_type']= ('City/Town')
pl_county['organization_type']= ('County')

In [13]:
pl_county['name'] = pl_county['name'].astype(str) + ' County' 

In [14]:
pl_county.ct_code.fillna(pl_county['unnamed:_8'], inplace=True)
pl_county = pl_county.drop(columns = {'unnamed:_8'})

In [15]:
plnames = pd.concat([pl_city, pl_county])

In [16]:
plnames.county.fillna(plnames.abbr, inplace=True)

In [17]:
fips_map = dict(zip(pl_county['abbr'], 
                          pl_county['co__fips']))

In [18]:
plnames['co__fips'] = plnames['county'].map(fips_map)

In [19]:
type_map = dict(zip(plnames['ct_code'], 
                          plnames['organization_type']))

In [20]:
dla['organization_type'] = dla['ct_code'].map(type_map)

In [24]:
mask1 = (dla['name'].str.contains('Council of Governments'))
mask2 = (dla['name'].str.contains('Tribe')) 
mask3 = (dla['name'].str.contains('University'))
mask4 = (dla['name'].str.contains('RTPA'))
mask5 = (dla['name'].str.contains('JPA'))

dla.loc[dla['organization_type'].isnull() & mask1, 'organization_type'] = 'Council of Governments'
dla.loc[dla['organization_type'].isnull() & mask2, 'organization_type'] = 'Tribe'
dla.loc[dla['organization_type'].isnull() & mask3, 'organization_type'] = 'University - Public'
dla.loc[dla['organization_type'].isnull() & mask4, 'organization_type'] = 'MPO/RTPA'
dla.loc[dla['organization_type'].isnull() & mask5, 'organization_type'] = 'Joint Powers Agency'

### READ ITP

In [28]:
itp = pd.read_csv(f"{GCS_FILE_PATH}organizations_All_Organizations.csv")

In [29]:
itp = to_snakecase(itp)

In [34]:
itp = itp>>select(_.name, _.ntp_id, _.itp_id, _.organization_type)

In [35]:
itp = itp.rename(columns = {"ntp_id":"ntd_id"})

## Merging

In [74]:
ids = pd.merge(itp, dla, on='name', how="outer", indicator=True)

In [75]:
ids._merge.value_counts()

right_only    924
left_only     711
both          117
Name: _merge, dtype: int64

* right only is DLA
* left only is ITP/NTD ID

In [76]:
ids>>count(_.name)>>arrange(-_.n)>>filter(_.n>1)

,name,n
187,Caltrans,13
1096,North East Trees,2
1650,Urban Corps of San Diego,2


In [77]:
ids>>filter(_.name=='North East Trees')

,name,ntd_id,itp_id,organization_type_x,ct_code,district,organization_type_y,_merge
1367,North East Trees,NaN,NaN,NaN,6118.0,7.0,NaN,right_only
1368,North East Trees,NaN,NaN,NaN,6434.0,7.0,NaN,right_only


In [78]:
ids>>filter(_.name=='Urban Corps of San Diego')

,name,ntd_id,itp_id,organization_type_x,ct_code,district,organization_type_y,_merge
1617,Urban Corps of San Diego,NaN,NaN,NaN,6490.0,11.0,NaN,right_only
1618,Urban Corps of San Diego,NaN,NaN,NaN,6233.0,11.0,NaN,right_only


In [79]:
#duplicates expected
ids>>filter(_.name=='Caltrans')

,name,ntd_id,itp_id,organization_type_x,ct_code,district,organization_type_y,_merge
845,Caltrans,NaN,NaN,NaN,6201.0,1.0,NaN,right_only
846,Caltrans,NaN,NaN,NaN,6202.0,2.0,NaN,right_only
847,Caltrans,NaN,NaN,NaN,6203.0,3.0,NaN,right_only
848,Caltrans,NaN,NaN,NaN,6204.0,4.0,NaN,right_only
849,Caltrans,NaN,NaN,NaN,6205.0,5.0,NaN,right_only
850,Caltrans,NaN,NaN,NaN,6206.0,6.0,NaN,right_only
851,Caltrans,NaN,NaN,NaN,6207.0,7.0,NaN,right_only
852,Caltrans,NaN,NaN,NaN,6208.0,8.0,NaN,right_only
853,Caltrans,NaN,NaN,NaN,6209.0,9.0,NaN,right_only
854,Caltrans,NaN,NaN,NaN,6210.0,10.0,NaN,right_only


In [80]:
#interesting that DLA Locode has some duplciates... will investigate
#caltrans duplicates expected

In [81]:
both = (ids>>filter(_._merge=='both'))

In [82]:
#both = to_snakecase(both)

In [83]:
both = both.drop(columns=['_merge'])

In [84]:
(
    (both.query("organization_type_x.isnull() or organization_type_y.isnull()"))
    >> select(_.organization_type_x, _.organization_type_y)
)

,organization_type_x,organization_type_y
4,NaN,NaN
16,NaN,NaN
28,Independent Agency,NaN
56,Joint Powers Agency,NaN
371,NaN,NaN
...,...,...
798,NaN,NaN
803,Independent Agency,NaN
815,Independent Agency,NaN
822,Independent Agency,NaN


In [85]:
both.organization_type_y.fillna(both['organization_type_x'], inplace=True)

In [86]:
#making sure there is no left over
both>>filter(_.organization_type_x.notnull())>>filter(_.organization_type_y.isnull())

,name,ntd_id,itp_id,organization_type_x,ct_code,district,organization_type_y


In [87]:
both = both.rename(columns={'organization_type_y':'organization_type'})
both.drop(columns=('organization_type_x'), inplace=True)

In [88]:
both

,name,ntd_id,itp_id,ct_code,district,organization_type
4,Access Services,90157,1.0,6312.0,7.0,NaN
13,Alpine County,9R02-91116,9.0,5931.0,10.0,County
15,Amador County,NaN,NaN,5926.0,10.0,County
16,Amador County Transportation Commission,NaN,NaN,6127.0,10.0,NaN
28,Antelope Valley Transit Authority,90121,16.0,6166.0,7.0,Independent Agency
...,...,...,...,...,...,...
798,Ventura County Transportation Commission,90164,380.0,6155.0,7.0,NaN
803,Victor Valley Transit Authority,90148,360.0,6261.0,8.0,Independent Agency
815,Western Contra Costa Transit Authority,90159,368.0,6226.0,4.0,Independent Agency
822,Yolo County Transportation District,90090,372.0,6195.0,3.0,Independent Agency


In [90]:
both.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117 entries, 4 to 824
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               117 non-null    object 
 1   ntd_id             56 non-null     object 
 2   itp_id             66 non-null     float64
 3   ct_code            117 non-null    float64
 4   district           117 non-null    float64
 5   organization_type  89 non-null     object 
dtypes: float64(3), object(3)
memory usage: 6.4+ KB


In [91]:
both['organization_type'].value_counts()

Independent Agency        41
County                    36
Council of Governments     7
Joint Powers Agency        3
MPO/RTPA                   1
City/Town                  1
Name: organization_type, dtype: int64

In [98]:
nomatch = ids>>filter(_._merge!='both')

In [99]:
nomatch.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1635 entries, 0 to 1751
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   name                 1635 non-null   object  
 1   ntd_id               143 non-null    object  
 2   itp_id               308 non-null    float64 
 3   organization_type_x  584 non-null    object  
 4   ct_code              924 non-null    float64 
 5   district             924 non-null    float64 
 6   organization_type_y  526 non-null    object  
 7   _merge               1635 non-null   category
dtypes: category(1), float64(3), object(4)
memory usage: 103.9+ KB


In [100]:
nomatch>>count(_._merge)

,_merge,n
0,left_only,711
1,right_only,924
2,both,0


In [101]:
(nomatch.query('itp_id.notnull() and ntd_id.notnull()')).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135 entries, 10 to 826
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   name                 135 non-null    object  
 1   ntd_id               135 non-null    object  
 2   itp_id               135 non-null    float64 
 3   organization_type_x  128 non-null    object  
 4   ct_code              0 non-null      float64 
 5   district             0 non-null      float64 
 6   organization_type_y  0 non-null      object  
 7   _merge               135 non-null    category
dtypes: category(1), float64(3), object(4)
memory usage: 8.7+ KB


In [103]:
nomatch.query('itp_id.isnull() or ntd_id.isnull() or ct_code.isnull()')

,name,ntd_id,itp_id,organization_type_x,ct_code,district,organization_type_y,_merge
0,A-Paratransit,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,ABC Shuttle,NaN,NaN,Company,NaN,NaN,NaN,left_only
2,Able Inc.,NaN,NaN,Non-Profit Organization,NaN,NaN,NaN,left_only
3,Abrazar Inc.,NaN,NaN,Non-Profit Organization,NaN,NaN,NaN,left_only
5,ACE Parking,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...
1747,Leave Blank,NaN,NaN,NaN,5465.0,53.0,NaN,right_only
1748,U.S. Fish and Wildlife Service,NaN,NaN,NaN,6250.0,53.0,NaN,right_only
1749,Tidewater Southern Railway Company,NaN,NaN,NaN,6031.0,53.0,NaN,right_only
1750,Tri-Counties Regional Park Group,NaN,NaN,NaN,6176.0,53.0,NaN,right_only


In [104]:
no_ids = nomatch>>filter(_.itp_id.isnull())>>filter(_.ntd_id.isnull())

In [106]:
no_ids>>filter(_.ct_code.isnull())

,name,ntd_id,itp_id,organization_type_x,ct_code,district,organization_type_y,_merge
0,A-Paratransit,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,ABC Shuttle,NaN,NaN,Company,NaN,NaN,NaN,left_only
2,Able Inc.,NaN,NaN,Non-Profit Organization,NaN,NaN,NaN,left_only
3,Abrazar Inc.,NaN,NaN,Non-Profit Organization,NaN,NaN,NaN,left_only
5,ACE Parking,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...
817,Westminster on Wheels Senior Transportation Pr...,NaN,NaN,NaN,NaN,NaN,NaN,left_only
818,Willits Seniors Inc.,NaN,NaN,Non-Profit Organization,NaN,NaN,NaN,left_only
820,Worldpay,NaN,NaN,Company,NaN,NaN,NaN,left_only
821,Yellow Cab CA,NaN,NaN,Company,NaN,NaN,NaN,left_only


In [107]:
both.query('itp_id.isnull() or ntd_id.isnull()')

,name,ntd_id,itp_id,ct_code,district,organization_type
15,Amador County,NaN,NaN,5926.0,10.0,County
16,Amador County Transportation Commission,NaN,NaN,6127.0,10.0,NaN
55,Butte County,NaN,NaN,5912.0,3.0,County
59,Calaveras Council of Governments,NaN,NaN,6128.0,10.0,Council of Governments
371,Colusa County Transportation Commission,NaN,NaN,6129.0,3.0,NaN
...,...,...,...,...,...,...
759,Transportation Agency for Monterey County,NaN,NaN,6143.0,5.0,NaN
765,Trinity County Transportation Commission,NaN,NaN,6153.0,2.0,NaN
768,Tulare County,NaN,NaN,5946.0,6.0,County
770,Tulare County Association of Governments,NaN,NaN,6094.0,6.0,Joint Powers Agency


In [108]:
both.query('organization_type.isnull()')

,name,ntd_id,itp_id,ct_code,district,organization_type
4,Access Services,90157,1.0,6312.0,7.0,NaN
16,Amador County Transportation Commission,NaN,NaN,6127.0,10.0,NaN
371,Colusa County Transportation Commission,NaN,NaN,6129.0,3.0,NaN
382,Council of San Benito County Governments,NaN,NaN,6060.0,5.0,NaN
408,El Dorado County Transportation Commission,NaN,NaN,6157.0,3.0,NaN
446,Glenn County Transportation Commission,NaN,NaN,6132.0,3.0,NaN
465,Humboldt County Association of Governments,NaN,NaN,6133.0,1.0,NaN
496,Kings County Association of Governments,NaN,NaN,6408.0,6.0,NaN
498,Lake County/City Area Planning Council,NaN,NaN,6136.0,1.0,NaN
500,Lassen County Transportation Commission,NaN,NaN,6137.0,2.0,NaN


In [109]:
# Most Agency types that are null are Transportation Commissions, or COGs. 

In [112]:
print(f"Out of {(len(ids>>filter(_.ct_code.notnull())))} organizations in the Official Locode List, {(len(ids>>filter(_.ct_code.notnull())>>filter(_.organization_type_x.notnull())))} have an Organization Type listed in Airtable")


Out of 1041 organizations in the Official Locode List, 85 have an Organization Type listed in Airtable


In [114]:
ids.organization_type_y.value_counts()

City/Town                 482
County                     58
University - Public        14
Council of Governments     12
MPO/RTPA                    2
Tribe                       1
Joint Powers Agency         1
Name: organization_type_y, dtype: int64

In [115]:
# There is a `Council of Governments` option available 

In [116]:
mask1 = (both['name'].str.contains('Council of Governments'))
mask2 = (both['name'].str.contains('Association of Governments')) 
mask4 = (both['name'].str.contains('Transportation Commission'))
mask5 = (both['name'].str.contains('Joint Powers Agency'))

both.loc[both['organization_type'].isnull() & mask1, 'organization_type'] = 'Council of Governments'
both.loc[both['organization_type'].isnull() & mask2, 'organization_type'] = 'Council of Governments'
both.loc[both['organization_type'].isnull() & mask4, 'organization_type'] = 'Transportation Commission'
both.loc[both['organization_type'].isnull() & mask5, 'organization_type'] = 'Joint Powers Agency'

In [117]:
mask6 = (both['name'].str.contains('Transportation Agency'))
mask8 = (both['name'].str.contains('Port Hueneme'))
mask9 = (both['name'].str.contains('Access Services'))
mask10 = (both['name'].str.contains('County Transportation Authority'))


both.loc[both['organization_type'].isnull() & mask6, 'organization_type'] = 'MPO/RTPA'
both.loc[both['organization_type'].isnull() & mask8, 'organization_type'] = 'City/Town'
both.loc[both['organization_type'].isnull() & mask9, 'organization_type'] = 'Independent Agency'
both.loc[both['organization_type'].isnull() & mask10, 'organization_type'] = 'Transportation Commission'

In [118]:
(both.query('organization_type.isnull()'))

,name,ntd_id,itp_id,ct_code,district,organization_type
382,Council of San Benito County Governments,NaN,NaN,6060.0,5.0,NaN
498,Lake County/City Area Planning Council,NaN,NaN,6136.0,1.0,NaN
608,Placer County Transportation Planning Agency,NaN,NaN,6158.0,3.0,NaN
694,Shasta County Regional Transportation Planning...,NaN,NaN,6093.0,2.0,NaN
739,Tahoe Regional Planning Agency,NaN,NaN,6125.0,3.0,NaN
774,Tuolumne County Transportation Council,NaN,NaN,6154.0,10.0,NaN


In [119]:
both>>filter(_.ntd_id.isnull())

,name,ntd_id,itp_id,ct_code,district,organization_type
15,Amador County,NaN,NaN,5926.0,10.0,County
16,Amador County Transportation Commission,NaN,NaN,6127.0,10.0,Transportation Commission
55,Butte County,NaN,NaN,5912.0,3.0,County
59,Calaveras Council of Governments,NaN,NaN,6128.0,10.0,Council of Governments
371,Colusa County Transportation Commission,NaN,NaN,6129.0,3.0,Transportation Commission
...,...,...,...,...,...,...
759,Transportation Agency for Monterey County,NaN,NaN,6143.0,5.0,MPO/RTPA
765,Trinity County Transportation Commission,NaN,NaN,6153.0,2.0,Transportation Commission
768,Tulare County,NaN,NaN,5946.0,6.0,County
770,Tulare County Association of Governments,NaN,NaN,6094.0,6.0,Joint Powers Agency


In [120]:
both>>filter(_.itp_id.isnull())

,name,ntd_id,itp_id,ct_code,district,organization_type
15,Amador County,NaN,NaN,5926.0,10.0,County
16,Amador County Transportation Commission,NaN,NaN,6127.0,10.0,Transportation Commission
55,Butte County,NaN,NaN,5912.0,3.0,County
59,Calaveras Council of Governments,NaN,NaN,6128.0,10.0,Council of Governments
371,Colusa County Transportation Commission,NaN,NaN,6129.0,3.0,Transportation Commission
382,Council of San Benito County Governments,NaN,NaN,6060.0,5.0,NaN
408,El Dorado County Transportation Commission,NaN,NaN,6157.0,3.0,Transportation Commission
433,Fresno Council of Governments,NaN,NaN,6504.0,6.0,Council of Governments
434,Fresno County,NaN,NaN,5942.0,6.0,County
446,Glenn County Transportation Commission,NaN,NaN,6132.0,3.0,Transportation Commission


In [121]:
# many in both have blank NTD and ITP ids but are still identified in Airtable

In [122]:
print(f"Out of the {(len(both))} organizations that matched, {(len(both.query('itp_id.notnull() and ntd_id.notnull() and organization_type.notnull()')))} have all id fields filled (ITP/NTD/Locode)")


Out of the 117 organizations that matched, 54 have all id fields filled (ITP/NTD/Locode)


In [123]:
#the 54 with all fields filled out
(both.query('itp_id.notnull() and ntd_id.notnull() and organization_type.notnull()'))

,name,ntd_id,itp_id,ct_code,district,organization_type
4,Access Services,90157,1.0,6312.0,7.0,Independent Agency
13,Alpine County,9R02-91116,9.0,5931.0,10.0,County
28,Antelope Valley Transit Authority,90121,16.0,6166.0,7.0,Independent Agency
56,Butte County Association of Governments,90208,47.0,6092.0,3.0,Joint Powers Agency
60,Calaveras County,9R02-91063,50.0,5930.0,10.0,County
407,El Dorado County Transit Authority,90229,101.0,6225.0,3.0,Independent Agency
435,Fresno County Rural Transit Agency,9R02-91007,117.0,6194.0,6.0,Independent Agency
445,Glenn County,9R02-91088,122.0,5911.0,3.0,County
451,Golden Empire Transit District,90004,126.0,6013.0,6.0,Independent Agency
468,Humboldt Transit Authority,9R02-91036,135.0,6162.0,1.0,Independent Agency


In [125]:
print(f"If we just look at completed Locodes (no ITP/NTD), we now have organization type info for {(len(both.query('ct_code.notnull() and organization_type.notnull()')))} organizations")


If we just look at completed Locodes (no ITP/NTD), we now have organization type info for 111 organizations


In [138]:
#both.to_parquet(f"{GCS_FILE_PATH}ITP_NTD_Locode_crosswalk.parquet")

#this just has the Agency Locode, NTD ID, ITP ID and Agency Locode

## Full Merged Data

In [126]:
ids

,name,ntd_id,itp_id,organization_type_x,ct_code,district,organization_type_y,_merge
0,A-Paratransit,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,ABC Shuttle,NaN,NaN,Company,NaN,NaN,NaN,left_only
2,Able Inc.,NaN,NaN,Non-Profit Organization,NaN,NaN,NaN,left_only
3,Abrazar Inc.,NaN,NaN,Non-Profit Organization,NaN,NaN,NaN,left_only
4,Access Services,90157,1.0,NaN,6312.0,7.0,NaN,both
...,...,...,...,...,...,...,...,...
1747,Leave Blank,NaN,NaN,NaN,5465.0,53.0,NaN,right_only
1748,U.S. Fish and Wildlife Service,NaN,NaN,NaN,6250.0,53.0,NaN,right_only
1749,Tidewater Southern Railway Company,NaN,NaN,NaN,6031.0,53.0,NaN,right_only
1750,Tri-Counties Regional Park Group,NaN,NaN,NaN,6176.0,53.0,NaN,right_only


In [129]:
print(f"Althought we still have {len(ids>>filter(_.organization_type_x.isnull()))} without Organization Types")

Althought we still have 1083 without Organization Types


In [131]:
ids>>filter(_.name.str.contains('Council'))>>select(_.name, _.organization_type_x, _.organization_type_y)

,name,organization_type_x,organization_type_y
59,Calaveras Council of Governments,Joint Powers Agency,Council of Governments
71,Cambria Community Council,Non-Profit Organization,NaN
382,Council of San Benito County Governments,NaN,NaN
433,Fresno Council of Governments,Joint Powers Agency,Council of Governments
492,Kern Council of Governments,NaN,NaN
498,Lake County/City Area Planning Council,NaN,NaN
532,Mendocino Council of Governments,NaN,Council of Governments
639,Sacramento Area Council of Governments,MPO/RTPA,Council of Governments
662,San Joaquin Council,NaN,NaN
663,San Joaquin Council of Governments,NaN,Council of Governments


In [132]:
ids.organization_type_y.fillna(ids['organization_type_x'], inplace=True)

In [133]:
ids = ids.rename(columns={'organization_type_y':'organization_type'})
ids.drop(columns=('organization_type_x'), inplace=True)

In [135]:
#all
ids

,name,ntd_id,itp_id,ct_code,district,organization_type,_merge
0,A-Paratransit,NaN,NaN,NaN,NaN,NaN,left_only
1,ABC Shuttle,NaN,NaN,NaN,NaN,Company,left_only
2,Able Inc.,NaN,NaN,NaN,NaN,Non-Profit Organization,left_only
3,Abrazar Inc.,NaN,NaN,NaN,NaN,Non-Profit Organization,left_only
4,Access Services,90157,1.0,6312.0,7.0,NaN,both
...,...,...,...,...,...,...,...
1747,Leave Blank,NaN,NaN,5465.0,53.0,NaN,right_only
1748,U.S. Fish and Wildlife Service,NaN,NaN,6250.0,53.0,NaN,right_only
1749,Tidewater Southern Railway Company,NaN,NaN,6031.0,53.0,NaN,right_only
1750,Tri-Counties Regional Park Group,NaN,NaN,6176.0,53.0,NaN,right_only
